<a href="https://colab.research.google.com/github/kwanglo/mge51101-20195171/blob/master/final_project/multisentiment_ML_W2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!pip3 install konlpy
!pip3 install soynlp

In [5]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, string
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [6]:
path='/gdrive/My Drive/Colab Notebooks/Final Project/dataset/'

train_data = pd.read_csv(path+"sentiment_train.csv")
valid_data = pd.read_csv(path+"sentiment_valid.csv")
test_data = pd.read_csv(path+"sentiment_test.csv")

In [ ]:
test_data.head()

In [7]:
train_x, train_y = train_data['Sentence'], train_data['Emotion']
valid_x, valid_y = valid_data['Sentence'], valid_data['Emotion']
test_x, test_y = test_data['Sentence'], test_data['Emotion']

In [8]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_data['Sentence'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)
xtest_count =  count_vect.transform(test_x)

In [9]:
#TF-IDF vectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=15000)
tfidf_vect.fit(train_data['Sentence'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
xtest_tfidf =  tfidf_vect.transform(test_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=15000)
tfidf_vect_ngram.fit(train_data['Sentence'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(test_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=15000)
tfidf_vect_ngram_chars.fit(train_data['Sentence'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [10]:
stop_words_set = pd.read_csv(path+'stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words)

In [11]:
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def preprocessing(text, okt, remove_stopwords = False, stop_words = []):
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    
    text_token = okt.morphs(text, stem = True)
    
    if remove_stopwords:
        text_token = [token for token in text_token if not token in stop_words]
        
    return text_token

In [12]:
#Preprocessing - Train and Valid data
okt = Okt()
clean_train_data = []
clean_valid_data = []
clean_test_data = []
for text in train_data['Sentence']:
    if type(text) == str:
        clean_train_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_train_data.append([])
        
for text in valid_data['Sentence']:
    if type(text) == str:
        clean_valid_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_valid_data.append([])
        
for text in test_data['Sentence']:
    if type(text) == str:
        clean_test_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_test_data.append([])

In [13]:
clean_train_data[:4]

[['대본', '없다', '도', '어버버', '하다', '걸리다', '닭'],
 ['거', '믿다', '진실', '은', '결국', '은', '거짓', '선동', '인가'],
 ['그러다가',
  '제대',
  '하다',
  '헤어지다',
  '이렇다',
  '은',
  '그냥',
  '휴가',
  '나오다',
  '한번',
  '하다',
  '이렇다',
  '용도',
  '라',
  '던데'],
 ['어제', '런닝맨', '완전', '재밌다']]

In [14]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_data)
train_sequences = tokenizer.texts_to_sequences(clean_train_data)
valid_sequences = tokenizer.texts_to_sequences(clean_valid_data)
test_sequences = tokenizer.texts_to_sequences(clean_test_data)

word_index = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 70

In [17]:
embeddings_index = {}
for i, line in enumerate(open(path+'wiki.ko.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
    
    
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
# 기존 모델
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return metrics.accuracy_score(predictions, valid_y)

In [25]:
# From other reference
def test_model(classifier, X_train, y_train, X_test):
  
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)

  return metrics.accuracy_score(y_pred, test_y)

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [27]:
# Naive Bayes on Count Vectors
valid_accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
test_accuracy = test_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xtest_count)

print("NB, Count Vectors Accuracy")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Naive Bayes on Word Level TF IDF Vectors
valid_accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
test_accuracy = test_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xtest_tfidf)
print("NB, WordLevel TF-IDF")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
valid_accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
test_accuracy = test_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xtest_tfidf_ngram)
print("NB, N-Gram Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Naive Bayes on Character Level TF IDF Vectors
valid_accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
test_accuracy = test_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xtest_tfidf_ngram_chars)
print("NB, CharLevel Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

NB, Count Vectors Accuracy
Validation Accuracy:  0.38013571869216534
Test Accuracy:  0.37424425634824665
NB, WordLevel TF-IDF
Validation Accuracy:  0.36964836520666255
Test Accuracy:  0.3653480739333218
NB, N-Gram Vectors
Validation Accuracy:  0.16681061073411474
Test Accuracy:  0.1731732596303334
NB, CharLevel Vectors
Validation Accuracy:  0.48834053053670573
Test Accuracy:  0.4920538953187079


In [31]:
# Linear Classifier on Count Vectors
valid_accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
test_accuracy = test_model(linear_model.LogisticRegression(), xtrain_count, train_y, xtest_count)
print("LR, Count Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Linear Classifier on Word Level TF IDF Vectors
valid_accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
test_accuracy = test_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xtest_tfidf)
print("NB, CharLevel Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
valid_accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
test_accuracy = test_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xtest_tfidf_ngram)

print("LR, N-Gram Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# Linear Classifier on Character Level TF IDF Vectors
valid_accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
test_accuracy = test_model(linear_model.LogisticRegression(), xtrain_count, train_y, xtest_count)
print("LR, CharLevel Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LR, Count Vectors
Validation Accuracy:  0.3731030228254164
Test Accuracy:  0.3667300051822422


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


NB, CharLevel Vectors
Validation Accuracy:  0.36557680444170265
Test Accuracy:  0.35792019347037485
LR, N-Gram Vectors
Validation Accuracy:  0.16693399136335596
Test Accuracy:  0.17256866470893073


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR, CharLevel Vectors
Validation Accuracy:  0.3731030228254164
Test Accuracy:  0.3667300051822422


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [33]:
# SVM on Ngram Level TF IDF Vectors
valid_accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
test_accuracy = test_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xtest_tfidf_ngram)
print("SVM, N-Gram Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

SVM, N-Gram Vectors
Validation Accuracy:  0.16594694632942628
Test Accuracy:  0.17127310416306787


In [36]:
# RF on Count Vectors
valid_accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
test_accuracy = test_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xtest_count)
print("RF, Count Vectors")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

# RF on Word Level TF IDF Vectors
valid_accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
test_accuracy = test_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xtest_tfidf)
print( "RF, WordLevel TF-IDF")
print("Validation Accuracy: ", valid_accuracy)
print("Test Accuracy: ", test_accuracy)

RF, Count Vectors
Validation Accuracy:  0.3068476249228871
Test Accuracy:  0.3117982380376576
RF, WordLevel TF-IDF
Validation Accuracy:  0.32387415175817397
Test Accuracy:  0.3186215235792019
